<a href="https://colab.research.google.com/github/broodfish/DCGAN/blob/master/dcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Upload files

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
os.chdir("gdrive/My Drive/HW2") 

In [0]:
# from google.colab import files
# uploaded = files.upload()

In [0]:
# !ls

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deep Convolutional Generative Adversarial Network

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/dcgan">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/dcgan.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/dcgan.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/generative/dcgan.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

### Import TensorFlow and other libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
import tensorflow as tf

In [0]:
# To generate GIFs
# !pip install imageio

In [0]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

### Load and prepare the dataset

You will use the MNIST dataset to train the generator and the discriminator. The generator will generate handwritten digits resembling the MNIST data.

In [0]:
import helper
from glob import glob
from PIL import Image
import math

In [0]:
data_dir = '../../../data'

In [0]:
helper.download_extract(data_dir)

In [0]:
images = []

for dirPath, dirNames, fileNames in os.walk(data_dir):
  for name in fileNames:
    images.append(os.path.join(dirPath, name))

In [0]:
n_samples = len(images)
image_list = np.empty((n_samples, 56, 56, 3), dtype = np.float32)
for i in range(0, n_samples):
  image = Image.open(images[i])
  image = image.resize((56, 56),Image.ANTIALIAS)
  image = np.asarray(image, dtype = np.float32)
  image = (image - 127.5) / 127.5 # Normalize the images to [-1, 1]
  image_list[i] = image

In [0]:
# print(type(image_list))
#　print(image_list.shape)

In [0]:
BUFFER_SIZE = 202599
BATCH_SIZE = 256

In [0]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(image_list).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [0]:
#print(type(train_dataset))
#print(train_dataset)

## Create the models

Both the generator and discriminator are defined using the [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model).

### The Generator

The generator uses `tf.keras.layers.Conv2DTranspose` (upsampling) layers to produce an image from a seed (random noise). Start with a `Dense` layer that takes this seed as input, then upsample several times until you reach the desired image size of 28x28x1. Notice the `tf.keras.layers.LeakyReLU` activation for each layer, except the output layer which uses tanh.

In [0]:
def make_generator_model():
    model = tf.keras.Sequential()
    # Input = (100,), Output = (7*7*256*3,) = (12544*3,)
    model.add(layers.Dense(7*7*256*3, use_bias = False, input_shape = (100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Input = (12544*3,), Output = (7, 7, 768)
    model.add(layers.Reshape((7, 7, 768)))
    assert model.output_shape == (None, 7, 7, 768) # Note: None is the batch size

    # Input = (7, 7, 768), Output = (7, 7, 384)
    model.add(layers.Conv2DTranspose(384, (5, 5), strides = (1, 1), padding = 'same', use_bias = False))
    assert model.output_shape == (None, 7, 7, 384)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Input = (7, 7, 384), Output = (14, 14, 192)
    model.add(layers.Conv2DTranspose(192, (5, 5), strides = (2, 2), padding = 'same', use_bias = False))
    assert model.output_shape == (None, 14, 14, 192)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Input = (14, 14, 192), Output = (28, 28, 96)
    model.add(layers.Conv2DTranspose(96, (5, 5), strides = (2, 2), padding = 'same', use_bias = False))
    assert model.output_shape == (None, 28, 28, 96)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Input = (28, 28, 96), Output = (56, 56, 3)
    model.add(layers.Conv2DTranspose(3, (5, 5), strides = (2, 2), padding = 'same', use_bias = False, activation = 'tanh'))
    assert model.output_shape == (None, 56, 56, 3)

    return model

Use the (as yet untrained) generator to create an image.

In [0]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training = False)

plt.imshow(generated_image[0, :, :, 0], cmap = 'gray')

### The Discriminator

The discriminator is a CNN-based image classifier.

In [0]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides = (2, 2), padding = 'same',
                                     input_shape = [56, 56, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides = (2, 2), padding = 'same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

Use the (as yet untrained) discriminator to classify the generated images as real or fake. The model will be trained to output positive values for real images, and negative values for fake images.

In [0]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

## Define the loss and optimizers

Define loss functions and optimizers for both models.


In [0]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [0]:
def discriminator_loss(true_label, fake_label, real_output, fake_output):
    real_loss = cross_entropy(true_label, real_output)
    fake_loss = cross_entropy(fake_label, fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, we will compare the discriminators decisions on the generated images to an array of 1s.

In [0]:
def generator_loss(true_label, fake_output):
    return cross_entropy(true_label, fake_output)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

### Save checkpoints
This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted.

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                 discriminator_optimizer = discriminator_optimizer,
                                 generator = generator,
                                 discriminator = discriminator)

## Define the training loop



In [0]:
EPOCHS = 1
noise_dim = 100
num_examples_to_generate = 9

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [0]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      # Train Discriminator
      generated_images = generator(noise, training = False)

      real_output = discriminator(images, training = True)
      fake_output = discriminator(generated_images, training = True)

      disc_true_label = tf.fill(real_output.shape, 0.9)
      disc_fake_label = tf.zeros_like(fake_output)
      disc_loss = discriminator_loss(disc_true_label, disc_fake_label, real_output, fake_output)

      # Train Generator
      generated_images = generator(noise, training = True)
      fake_output = discriminator(generated_images, training = False)
      gen_true_label = tf.fill(fake_output.shape, 0.9)
      gen_loss = generator_loss(gen_true_label, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [0]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    gen_losses = []
    disc_losses = []
    generator
    for image_batch in dataset:
      gen_loss, disc_loss = train_step(image_batch)
      gen_losses.append(gen_loss)
      disc_losses.append(disc_loss)


    # Produce images for the GIF as we go
    display.clear_output(wait = True)
    generate_and_save_images(generator, epoch + 1, seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 5 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    mean_gen_losses = tf.reduce_mean(gen_losses)
    mean_disc_losses = tf.reduce_mean(disc_losses)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))
    print('Generator loss:{}'.format(mean_gen_losses))
    print('Discriminator loss:{}'.format(mean_disc_losses))

  # Generate after the final epoch
  display.clear_output(wait = True)
  generate_and_save_images(generator, epochs, seed)

**Generate and save images**



In [0]:
def output_fig(images_array, file_name):
    plt.figure(figsize=(6, 6), dpi=100)
    # Get maximum size for square grid of images
    save_size = math.floor(np.sqrt(images_array.shape[0]))

    # Scale to 0-255
    images_array = tf.cast(images_array * 127.5 + 127.5, dtype = tf.uint8)

    # Put images in a square arrangement
    images_in_square = np.reshape(
            images_array[:save_size*save_size],
            (save_size, save_size, images_array.shape[1], images_array.shape[2], images_array.shape[3]))

    # Combine images to grid image
    new_im = Image.new('RGB', (images_array.shape[1] * save_size, images_array.shape[2] * save_size))
    for col_i, col_images in enumerate(images_in_square):
        for image_i, image in enumerate(col_images):
            im = Image.fromarray(image, 'RGB')
            new_im.paste(im, (col_i * images_array.shape[1], image_i * images_array.shape[2]))
    plt.imshow(new_im)
    plt.axis("off")
    plt.savefig(file_name + '.png', bbox_inches = 'tight', pad_inches = 0)    
    #plt.show()

In [0]:
#@title Default title text
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training = False)
  output_fig(predictions, "./result/results_1_{:02d}".format(epoch))


## Train the model
Call the `train()` method defined above to train the generator and discriminator simultaneously. Note, training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

At the beginning of the training, the generated images look like random noise. As training progresses, the generated digits will look increasingly real. After about 50 epochs, they resemble MNIST digits. This may take about one minute / epoch with the default settings on Colab.

Restore the latest checkpoint.

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
%%time
train(train_dataset, EPOCHS)

In [0]:
checkpoint.save(file_prefix = checkpoint_prefix)

In [0]:
for i in range(500):
    generated_im = tf.random.normal([num_examples_to_generate, noise_dim])
    result = generator(generated_im, training = False)
    # print(result.shape) # should be (9, width, height, 3)
    output_fig(result, file_name = "./images/{}_image".format(str.zfill(str(i), 3)))

## Create a GIF


In [0]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('./results_1_{:02d}.png'.format(epoch_no))

In [0]:
display_image(EPOCHS)

Use `imageio` to create an animated gif using the images saved during training.

In [0]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('results*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename = anim_file)

If you're working in Colab you can download the animation with the code below:

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(anim_file)

## Next steps


This tutorial has shown the complete code necessary to write and train a GAN. As a next step, you might like to experiment with a different dataset, for example the Large-scale Celeb Faces Attributes (CelebA) dataset [available on Kaggle](https://www.kaggle.com/jessicali9530/celeba-dataset/home). To learn more about GANs we recommend the [NIPS 2016 Tutorial: Generative Adversarial Networks](https://arxiv.org/abs/1701.00160).
